In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta, datetime, date
import os
from utils import data_paths, load_config
from pathlib import Path
from nltk.metrics import edit_distance #(Levenshtein)
import pycountry
import math

In [3]:
LOCAL_FILES=True
#jupyter or script
IS_SCRIPT = False

In [4]:
os.getcwd()

'/mnt/963GB/Data/Python/ACode/medical/covid19/forecaster/covidforecaster/tools'

In [5]:
if IS_SCRIPT:
    RUN_PATH = Path(os.path.realpath(__file__))
    DATA_PARENT = RUN_PATH.parent.parent
else:
    #for jupyter
    cw=!pwd
    RUN_PATH = Path(cw[0])
    DATA_PARENT = RUN_PATH.parent

In [6]:
if IS_SCRIPT:
    csse_data = data_paths('tools/csse_data_paths.yml')
else:
    csse_data = data_paths('csse_data_paths.yml')

In [7]:
if LOCAL_FILES:
    confirmed_url=csse_data.get("csse_ts_local", {}).get('confirmed', {})
    deaths_url=csse_data.get("csse_ts_local", {}).get('deaths', {})
    recovered_url=csse_data.get("csse_ts_local", {}).get('recovered', {})
    
    confirmed_url = str(DATA_PARENT/confirmed_url)
    deaths_url = str(DATA_PARENT/deaths_url)
    recovered_url = str(DATA_PARENT/recovered_url)
else:
    confirmed_url=csse_data.get("csse_ts_global", {}).get('confirmed', {})
    deaths_url=csse_data.get("csse_ts_global", {}).get('deaths', {})
    recovered_url=csse_data.get("csse_ts_global", {}).get('recovered', {})

In [8]:
### UN stats

In [9]:
df_un_pop_density_info=pd.read_csv(DATA_PARENT/'data/un/df_un_pop_density_info.csv')
df_un_urban_growth_info=pd.read_csv(DATA_PARENT/'data/un/urban_growth_info.csv')
df_un_health_info=pd.read_csv(DATA_PARENT/'data/un/df_un_health_info.csv')
df_un_tourism_info=pd.read_csv(DATA_PARENT/'data/un/df_un_tourism_info.csv')
df_un_gdp_info=pd.read_csv(DATA_PARENT/'data/un/df_un_gdp_info.csv')
df_un_edu_info=pd.read_csv(DATA_PARENT/'data/un/df_un_edu_info.csv')
df_un_pop_growth_info=pd.read_csv(DATA_PARENT/'data/un/df_un_pop_growth_info.csv')
df_un_gdrp_rnd_info=pd.read_csv(DATA_PARENT/'data/un/df_un_gdrp_rnd_info.csv')
df_un_education_info=pd.read_csv(DATA_PARENT/'data/un/df_un_education_info.csv')
df_un_sanitation_info=pd.read_csv(DATA_PARENT/'data/un/df_un_sanitation_info.csv')

df_un_health_expenditure_info=pd.read_csv(DATA_PARENT/'data/un/df_un_health_expenditure_info.csv')
df_un_immigration_info=pd.read_csv(DATA_PARENT/'data/un/df_un_immigration_info.csv')
df_un_trading_info=pd.read_csv(DATA_PARENT/'data/un/df_un_trading_info.csv')
df_un_land_info=pd.read_csv(DATA_PARENT/'data/un/df_un_land_info.csv')

In [105]:
df_un_gdp_info.head()

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source
0,4,Afghanistan,1985,GDP in current prices (millions of US dollars),3322.0,NaN,"United Nations Statistics Division, New York, ..."
1,4,Afghanistan,1995,GDP in current prices (millions of US dollars),3236.0,NaN,"United Nations Statistics Division, New York, ..."
2,4,Afghanistan,2005,GDP in current prices (millions of US dollars),6622.0,NaN,"United Nations Statistics Division, New York, ..."
3,4,Afghanistan,2010,GDP in current prices (millions of US dollars),16078.0,NaN,"United Nations Statistics Division, New York, ..."
4,4,Afghanistan,2015,GDP in current prices (millions of US dollars),20608.0,NaN,"United Nations Statistics Division, New York, ..."


In [106]:
df_un_gdrp_rnd_info.head()

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,8,Albania,2008,Gross domestic expenditure on R & D: as a perc...,0.2,Partial data.,"United Nations Educational, Scientific and Cul...",NaN,NaN,NaN
1,8,Albania,2008,Gross domestic expenditure on R & D: Business ...,3.3,Partial data.,"United Nations Educational, Scientific and Cul...",NaN,NaN,NaN
2,8,Albania,2008,Gross domestic expenditure on R & D: Governmen...,80.8,Partial data.,"United Nations Educational, Scientific and Cul...",NaN,NaN,NaN
3,8,Albania,2008,Gross domestic expenditure on R & D: Higher ed...,8.6,Partial data.,"United Nations Educational, Scientific and Cul...",NaN,NaN,NaN
4,8,Albania,2008,Gross domestic expenditure on R & D: Funds fro...,7.4,Partial data.,"United Nations Educational, Scientific and Cul...",NaN,NaN,NaN


In [51]:
df_un_health_info.head()
#Health personnel: Pharmacists (per 1000 population)

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source
0,4,Afghanistan,2001,Health personnel: Physicians (number),4104.0,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
1,4,Afghanistan,2001,Health personnel: Physicians (per 1000 populat...,0.2,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
2,4,Afghanistan,2001,Health personnel: Pharmacists (number),525.0,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
3,4,Afghanistan,2001,Health personnel: Pharmacists (per 1000 popula...,0.0,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
4,4,Afghanistan,2005,Health personnel: Pharmacists (number),900.0,NaN,"World Health Organisation (WHO), Geneva, WHO G..."


In [91]:
df_physicians=df_un_health_info.loc[df_un_health_info['Series'] == 'Health personnel: Physicians (per 1000 population)']
df_physicians=df_physicians.loc[df_physicians.groupby('Country')['Year'].idxmax()]

In [92]:
df_physicians.head()

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source
41,4,Afghanistan,2016,Health personnel: Physicians (per 1000 populat...,0.3,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
59,8,Albania,2016,Health personnel: Physicians (per 1000 populat...,1.2,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
85,12,Algeria,2016,Health personnel: Physicians (per 1000 populat...,1.8,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
95,20,Andorra,2015,Health personnel: Physicians (per 1000 populat...,3.3,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
115,24,Angola,2017,Health personnel: Physicians (per 1000 populat...,0.2,NaN,"World Health Organisation (WHO), Geneva, WHO G..."


In [93]:
df_physicians.rename(columns={'Value': 'Physicians_per_1000_pop'}, inplace=True)
df_physicians.drop(columns=['Region/Country/Area','Year','Series','Footnotes','Source'],inplace=True)

In [94]:
df_physicians.head()

,Country,Physicians_per_1000_pop
41,Afghanistan,0.3
59,Albania,1.2
85,Algeria,1.8
95,Andorra,3.3
115,Angola,0.2


In [ ]:
#population density

In [69]:
df_population_density=df_un_pop_density_info.loc[df_un_pop_density_info['Series'] == 'Population density']

In [70]:
df_population_density.tail(n=50)
#Population aged 60+ years old (percentage)
#Population density
#Population mid-year estimates (millions)

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source,Unnamed: 7
4417,788,Tunisia,2017,Population density,73.5932,NaN,"United Nations Population Division, New York, ...",NaN
4425,788,Tunisia,2019,Population density,75.2750,NaN,"United Nations Population Division, New York, ...",NaN
4432,792,Turkey,2005,Population density,88.2287,NaN,"United Nations Population Division, New York, ...",NaN
4439,792,Turkey,2010,Population density,93.9763,NaN,"United Nations Population Division, New York, ...",NaN
4446,792,Turkey,2017,Population density,105.3967,NaN,"United Nations Population Division, New York, ...",NaN
4454,792,Turkey,2019,Population density,108.4022,NaN,"United Nations Population Division, New York, ...",NaN
4461,800,Uganda,2005,Population density,138.5546,NaN,"United Nations Population Division, New York, ...",NaN
4468,800,Uganda,2010,Population density,162.2950,NaN,"United Nations Population Division, New York, ...",NaN
4475,800,Uganda,2017,Population density,206.0287,NaN,"United Nations Population Division, New York, ...",NaN
4483,800,Uganda,2019,Population density,221.5585,NaN,"United Nations Population Division, New York, ...",NaN


In [73]:

df_population_density=df_population_density.loc[df_population_density.groupby('Country')['Year'].idxmax()]

In [74]:

#df_population_density.to_csv(DATA_PARENT/'data/un/df_un_population_density_stats.csv', index=False)

In [75]:
df_population_60_plus=df_un_pop_density_info.loc[df_un_pop_density_info['Series'] == 'Population aged 60+ years old (percentage)']

In [76]:
df_population_60_plus=df_population_60_plus.loc[df_population_60_plus.groupby('Country')['Year'].idxmax()]

In [87]:
df_population_60_plus.head()

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source,Unnamed: 7
27,4,Afghanistan,2019,Population aged 60+ years old (percentage),4.1655,NaN,"United Nations Population Division, New York, ...",NaN
56,8,Albania,2019,Population aged 60+ years old (percentage),20.4779,NaN,"United Nations Population Division, New York, ...",NaN
85,12,Algeria,2019,Population aged 60+ years old (percentage),9.6817,NaN,"United Nations Population Division, New York, ...",NaN
100,20,Andorra,2016,Population aged 60+ years old (percentage),19.0387,De jure population.,Population statistics are compiled from regist...,"United Nations Population Division, New York, ..."
134,24,Angola,2019,Population aged 60+ years old (percentage),3.6243,NaN,"United Nations Population Division, New York, ...",NaN


In [88]:
df_population_60_plus.rename(columns={'Value': 'Population_60+'}, inplace=True)
df_population_60_plus.drop(columns=['Region/Country/Area','Year','Series','Footnotes','Source','Unnamed: 7'],inplace=True)

In [89]:
df_population_60_plus.head()

,Country,Population_60+
27,Afghanistan,4.1655
56,Albania,20.4779
85,Algeria,9.6817
100,Andorra,19.0387
134,Angola,3.6243


In [77]:
df_pop_estimate=df_un_pop_density_info.loc[df_un_pop_density_info['Series'] =='Population mid-year estimates (millions)']

In [78]:
df_pop_estimate=df_pop_estimate.loc[df_pop_estimate.groupby('Country')['Year'].idxmax()]

In [84]:
df_pop_estimate.head()

,Region/Country/Area,Country,Year,Series,Value,Footnotes,Source,Unnamed: 7
22,4,Afghanistan,2019,Population mid-year estimates (millions),38.0418,NaN,"United Nations Population Division, New York, ...",NaN
51,8,Albania,2019,Population mid-year estimates (millions),2.8809,NaN,"United Nations Population Division, New York, ...",NaN
80,12,Algeria,2019,Population mid-year estimates (millions),43.0531,NaN,"United Nations Population Division, New York, ...",NaN
105,20,Andorra,2019,Population mid-year estimates (millions),0.0771,NaN,"United Nations Population Division, New York, ...",NaN
129,24,Angola,2019,Population mid-year estimates (millions),31.8253,NaN,"United Nations Population Division, New York, ...",NaN


In [85]:
df_pop_estimate.rename(columns={'Value': 'Population_million'}, inplace=True)
df_pop_estimate.drop(columns=['Region/Country/Area','Year','Series','Footnotes','Source','Unnamed: 7'],inplace=True)

In [86]:
df_pop_estimate.head()

,Country,Population_million
22,Afghanistan,38.0418
51,Albania,2.8809
80,Algeria,43.0531
105,Andorra,0.0771
129,Angola,31.8253


In [80]:
df_population_density.rename(columns={'Value': 'Population_density'}, inplace=True)
df_population_density.drop(columns=['Region/Country/Area','Year','Footnotes','Source','Unnamed: 7','Series'],inplace=True)

In [83]:
df_population_density.head()

,Country,Population_density
28,Afghanistan,58.2694
57,Albania,105.1430
86,Algeria,18.0763
106,Andorra,164.1319
135,Angola,25.5276


In [100]:
df_un_merged = pd.merge(df_pop_estimate, df_population_density)

In [101]:
df_un_merged = pd.merge(df_un_merged, df_population_60_plus)

In [103]:
df_un_merged = pd.merge(df_un_merged, df_physicians)

In [104]:
df_un_merged.head()

,Country,Population_million,Population_density,Population_60+,Physicians_per_1000_pop
0,Afghanistan,38.0418,58.2694,4.1655,0.3
1,Albania,2.8809,105.1430,20.4779,1.2
2,Algeria,43.0531,18.0763,9.6817,1.8
3,Andorra,0.0771,164.1319,19.0387,3.3
4,Angola,31.8253,25.5276,3.6243,0.2


In [107]:
df_un_merged.to_csv(DATA_PARENT/'data/un/df_un_merged_stats.csv', index=False)

In [10]:
### Freedom House stats

In [108]:
#Freedon House stats
def country_freedom():
    global_freedom = str(DATA_PARENT/'data/freedom_house/Global_Freedom.csv')
    df_global_free = pd.read_csv(global_freedom)
    internet_freedom = str(DATA_PARENT/'data/freedom_house/Internet_Freedom.csv')
    df_internet_free = pd.read_csv(internet_freedom)
    return df_global_free, df_internet_free
df_global_freedom, df_internet_freedom = country_freedom()

In [109]:
df_global_freedom.head()

,Country,Toral Score,Status,Political Rights,Civil Liberties
0,Netherlands,99,Free,40,59
1,Canada,98,Free,40,58
2,Luxembourg,98,Free,38,60
3,Uruguay,98,Free,40,58
4,Australia,97,Free,40,57


In [111]:
df_global_freedom.rename(columns={'Toral Score': 'Global_Score', 'Status': 'Global_Status'}, inplace=True)

In [112]:
df_internet_freedom.rename(columns={'Total Score': 'Internet_Score', 'Status': 'Internet_Status', 'Obstacles to Access': 'Internet_Access_Obstacles', 'Limits on Content': 'Internet_Content_Limit', 'Violations of User Rights': 'Internet_User_Rights_Violations'}, inplace=True)

In [113]:
df_global_freedom.head()

,Country,Global_Score,Global_Status,Political Rights,Civil Liberties
0,Netherlands,99,Free,40,59
1,Canada,98,Free,40,58
2,Luxembourg,98,Free,38,60
3,Uruguay,98,Free,40,58
4,Australia,97,Free,40,57


In [114]:
df_freedomhouse_merged = pd.merge(df_global_freedom, df_internet_freedom)

In [115]:
df_freedomhouse_merged.to_csv(DATA_PARENT/'data/freedom_house/df_freedomhouse_merged.csv', index=False)